# ETL & DATA CLEANING : Prédiction de la Gravité des Accidents Routiers

## Introduction

Ce notebook effectue un processus complet de **nettoyage et préparation des données (ETL)** à partir des bases de données BAAC (Bulletin d'Analyse d'Accidents de la circulation). Disponible sur le site suivant: https://www.data.gouv.fr/datasets/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2024

**Objectif final** : Produire un dataset clean, encodé et prêt pour la modélisation ML.

**Sources de données** (2021-2024) :
- **Caractéristiques (caract)** : Contexte de l'accident (jour, heure, météo, etc.)
- **Lieux (lieux)** : Localisation géographique et type de route
- **Véhicules (vehicules)** : Infos sur le véhicule impliqué
- **Usagers (usagers)** : **Table centrale (de fait)** - chaque ligne = une personne impliquée

**Structure du pipeline** :   

* Fusion Relationnelle : Agrégation de 4 ans de données (2021-2024) et jointures horizontales centrées sur la table Usagers.

* Nettoyage de Structure : Suppression des colonnes vides à plus de 70 %, retrait des données de localisation trop fines (PR/PR1) et élimination des doublons.

* Prévention du Leakage : Suppression des variables connues uniquement après l'accident pour garantir l'intégrité prédictive du modèle.

* Feature Engineering : Création de variables à fort signal : âge, moment de la journée (nuit/jour), weekend et météo dégradée.

* Finalisation : Imputation des valeurs manquantes (médiane/mode) et encodage final (One-Hot Encoding) pour rendre le dataset exploitable par une IA.


# 1. Import des bibliothèques

In [1]:
import pandas as pd
import numpy as np
import os



# 2. Préparation et agrégation des données

In [2]:
# Configuration des répertoires
raw_path = "../data/raw/"
processed_path = "../data/processed/"
os.makedirs(processed_path, exist_ok=True)

## 2.1 Concaténation (empilement vertical des années)   
Les données sont fragmentées par année (2021-2024). Pour une analyse globale, il faut regrouper ces fichiers pour chaque thématique (Usagers, Véhicules, etc.).   
Stratégie : Nous créons une fonction qui parcourt les années, lit les fichiers correspondants et les "empile" verticalement. Cette séparation permet d'ajouter facilement de nouvelles années au projet sans modifier la logique de fusion.

In [3]:
years = [2021, 2022, 2023, 2024]

def stack_years(category):
    dfs = []
    for y in years:
        file = os.path.join(raw_path, f"{category}-{y}.csv")
        if os.path.exists(file):
            dfs.append(pd.read_csv(file, sep=';', encoding='utf-8', low_memory=False))
    return pd.concat(dfs, ignore_index=True)

# Centralisation
usagers_all = stack_years("usagers")
veh_all     = stack_years("vehicules")
caract_all  = stack_years("caract")
lieux_all   = stack_years("lieux")

print(f"Historique Usagers : {len(usagers_all)} lignes.")

Historique Usagers : 506886 lignes.


## 2.2 Jointure relationnelle  
Les données BAAC sont morcelées en quatre fichiers annuels (Caractéristiques, Lieux, Véhicules, Usagers). Une fusion maladroite peut entraîner une perte de données ou une duplication artificielle des lignes.   
 *Stratégie* : La table Usagers est notre table de fait centrale car la variable cible grav. Nous effectuons une jointure "Left Join" pour conserver tous les usagers et leur associer les caractéristiques de l'accident et du véhicule. La table lieux est dé-dupliquée au préalable pour garantir qu'un accident ne pointe que vers une seule localisation géographique.

In [4]:
lieux_unique = lieux_all.drop_duplicates(subset='Num_Acc')

df = pd.merge(usagers_all, veh_all, on=['Num_Acc', 'id_vehicule'], how='left', suffixes=('', '_v'))
df = pd.merge(df, caract_all, on='Num_Acc', how='left')
df = pd.merge(df, lieux_unique, on='Num_Acc', how='left')

print(f"Dataset fusionné : {df.shape[0]} lignes.")

Dataset fusionné : 506886 lignes.


## 3. Nettoyage des données (Data Cleaning)  

### 3.1 Gestion des valeurs "Non Renseignées"
La documentation indique que les valeurs -1, 0 (pour certaines colonnes) ou les points "." correspondent à des données non renseignées. Pour que le modèle ne les considère pas comme des nombres réels, il faut les convertir en NaN.   
Certaines variables sont trop vides ou trop précises pour être généralisables par une IA.   
Stratégie :   
 1. Seuil de 70% : Nous supprimons les colonnes comme v1 ou secu3 qui sont vides à plus de 70%. Cela évite d'introduire du bruit par une imputation massive.   
 2. Suppression de PR/PR1 : Ces marqueurs kilométriques sont trop spécifiques. Les retirer force le modèle à apprendre des facteurs globaux (vitesse, météo) plutôt que des coordonnées précises, améliorant ainsi sa capacité de généralisation.

In [5]:
# Inspection avant nettoyage 

# 1. On définit ce qu'on cherche
valeurs_vides = [-1, 0, '-1', '0', '.', '0.0']

# 2. On crée un masque : True si la cellule contient une valeur "vide", False sinon
masque_vides = df.isin(valeurs_vides)

# 3. On compte par colonne (en ignorant Num_Acc et grav pour l'instant)
compte_vides = masque_vides.drop(columns=['Num_Acc', 'grav']).sum().sort_values(ascending=False)

# 4. On calcule le pourcentage
pourcentage_vides = (compte_vides / len(df) * 100).round(2)

# 5. On affiche un résumé
print("ANALYSE DES DONNÉES NON RENSEIGNÉES (AVANT NETTOYAGE)")
print("-" * 50)
resume_vides = pd.DataFrame({
    'Nb valeurs "vides"': compte_vides,
    '% du total': pourcentage_vides
})

# On ne montre que les colonnes qui ont au moins une valeur vide
display(resume_vides[resume_vides['Nb valeurs "vides"'] > 0].head(15))

# 6. Échantillon des lignes "à problèmes"
print("\nAPERÇU DE 5 LIGNES CONTENANT DES VALEURS -1 ou 0 :")
# On cherche les lignes qui ont au moins un "vide" dans n'importe quelle colonne
lignes_a_problemes = df[masque_vides.any(axis=1)]
display(lignes_a_problemes.head(5))

ANALYSE DES DONNÉES NON RENSEIGNÉES (AVANT NETTOYAGE)
--------------------------------------------------


,"Nb valeurs ""vides""",% du total
v1,505991,99.82
secu3,501513,98.94
locp,467730,92.28
etatp,467713,92.27
vosp,447810,88.35
infra,430410,84.91
obs,424706,83.79
secu2,404723,79.84
actp,229965,45.37
trajet,144900,28.59



APERÇU DE 5 LIGNES CONTENANT DES VALEURS -1 ou 0 :


,Num_Acc,id_usager,id_vehicule,num_veh,place,catu,grav,sexe,an_nais,trajet,...,prof,pr,pr1,plan,lartpc,larrout,surf,infra,situ,vma
0,202100000001,267 638,201 764,B01,1,1,3,1,2000.0,1,...,1,(1),(1),1,NaN,-1,1,0,1,80
1,202100000001,267 639,201 765,A01,1,1,1,1,1978.0,1,...,1,(1),(1),1,NaN,-1,1,0,1,80
2,202100000002,267 636,201 762,A01,1,1,4,1,1983.0,0,...,1,0,10,1,NaN,-1,1,0,1,80
3,202100000002,267 637,201 763,B01,1,1,3,1,1993.0,0,...,1,0,10,1,NaN,-1,1,0,1,80
4,202100000003,267 634,201 761,A01,1,1,1,1,1995.0,1,...,1,(1),(1),1,NaN,-1,1,0,1,50


In [6]:
# Remplacement des codes métier (-1, 0, .) par NaN
df = df.replace([-1, 0, "-1", "0", ".", "0.0"], np.nan)

# 1. Seuil de 70% de vides
seuil = 0.70
limit = len(df) * (1 - seuil)
df = df.dropna(thresh=limit, axis=1)

# 2. Suppression PR/PR1 (trop précis)
df = df.drop(columns=['pr', 'pr1'], errors='ignore')

# Nettoyage de la cible
df = df.dropna(subset=['grav'])
df['grav'] = df['grav'].astype(int)

## 3.2. Gestion du Leakage et Doublons
Les doublons et les variables connues "après coup" (leakage) faussent les performances du modèle.   
Stratégie : Nous supprimons les doublons sur le couple (Num_Acc, id_usager). Nous retirons aussi les variables post-accident (actp, etatp, locp, secu2) car elles ne sont pas disponibles pour prédire un accident avant qu'il n'arrive.

In [7]:
# Doublons
df = df.drop_duplicates(subset=['Num_Acc', 'id_usager'])

# Data Leakage Prevention
leak_cols = ['actp', 'etatp', 'locp', 'secu2', 'secu3']
df = df.drop(columns=[c for c in leak_cols if c in df.columns], errors='ignore')

# 4. Feature Engineering    
Les données brutes (heure, année naissance) sont peu digestes. Les trous restants doivent être comblés.   
Stratégie (Création de variables à fort signal ):

* Âge (vulnérabilité).

* Weekend (comportements de loisirs).

* Is_work_trip (stress lié au travail vs loisirs).

* Meteo_degradee (conditions d'adhérence).

* Moment_journee (visibilité).

In [8]:
# 1. Âge : Reflète la vulnérabilité physique de l'usager
df['age'] = 2024 - df['an_nais']

# 2. Moment de la journée : Influence de la visibilité et de la fatigue
def get_moment_journee(val):
    # 1. Gestion des valeurs manquantes (NaN)
    if pd.isna(val):
        return 'inconnu'
    
    try:
        # 2. Extraction de l'heure
        # On gère le format '12:30' (split sur :) ou '1230' (HHMM)
        val_str = str(val).strip()
        
        if ':' in val_str:
            h = int(val_str.split(':')[0])
        elif len(val_str) >= 3:
            # Cas HHMM (ex: 1230 -> 12)
            h = int(val_str[:-2])
        else:
            # Cas heure simple
            h = int(val_str)
            
        # 3. Logique Nuit (22h-6h) vs Jour
        if h < 6 or h > 21:
            return 'nuit'
        return 'jour'
        
    except (ValueError, TypeError, IndexError):
        # En cas de format de texte inattendu
        return 'inconnu'

# Application de la fonction
df['moment_journee'] = df['hrmn'].apply(get_moment_journee)

# Vérification
#print(df['moment_journee'].value_counts())

# 3. Extraction du Weekend : Comportements de loisirs vs semaine
df['temp_date'] = pd.to_datetime(df[['jour', 'mois', 'an']].rename(columns={'jour': 'day', 'mois': 'month', 'an': 'year'}), errors='coerce')
df['is_weekend'] = df['temp_date'].dt.weekday.apply(lambda x: 1 if x >= 5 else 0)

# 4. Is_work_trip : Stress lié au travail vs Loisirs
# Stratégie : On regroupe les modalités 1 (Domicile-Travail) et 4 (Usage professionnel) 
# pour isoler le risque routier professionnel.
def map_work_trip(x):
    try:
        x = int(x)
        if x in [1, 4]: # 1: Domicile-Travail, 4: Professionnel
            return 1
        return 0 # Loisirs, courses, école, etc.
    except:
        return 0

df['is_work_trip'] = df['trajet'].apply(map_work_trip)

# 5. Météo Dégradée : Simplification du signal atmosphérique (atm)
def map_meteo(x):
    try:
        x = int(x)
        if x in [3, 4, 5]: # 3: Pluie forte, 4: Neige/Grêle, 5: Brouillard
            return 1
        return 0
    except:
        return 0

df['meteo_degradee'] = df['atm'].apply(map_meteo)

print("Vérification des nouvelles variables :")
print(f"- Trajets travail : {df['is_work_trip'].mean():.2%}")
print(f"- Accidents weekend : {df['is_weekend'].mean():.2%}")

Vérification des nouvelles variables :
- Trajets travail : 21.93%
- Accidents weekend : 20.71%


# 5. Nettoyage avancé et export
Les algorithmes de ML ne supportent pas les NaN restants ni les variables textuelles. 
Stratégie : 
1. Imputation : Médiane pour l'âge (robuste) et Mode pour les catégories. 
2. Bruit : Suppression des identifiants techniques et des colonnes redondantes. 
3. Encodage : Utilisation de get_dummies (One-Hot Encoding) pour transformer les textes en chiffres.

In [9]:
# 1. Suppression des identifiants et colonnes toxiques
to_drop_final = ['Num_Acc', 'id_usager', 'id_vehicule', 'num_veh', 'num_veh_v', 'adr', 'temp_date']
df = df.drop(columns=[c for c in to_drop_final if c in df.columns], errors='ignore')

# 2. Filtre de sécurité automatique (Cardinalité)
# Supprime toute colonne texte restante qui dépasse 100 modalités
for col in df.select_dtypes(include=['object']).columns:
    n_unique = df[col].nunique()
    if n_unique > 100:
        print(f"Suppression de sécurité : '{col}' ({n_unique} valeurs uniques)")
        df = df.drop(columns=[col])

# 3. Encodage One-Hot
# Transforme les variables qualitatives (ex: moment_journee) en nombres 0/1
df_final = pd.get_dummies(df, drop_first=True)

# 4. Conversion float32 pour l'économie de mémoire
# Réduit le poids du fichier final sans perdre d'information prédictive
df_final = df_final.astype('float32')

print(f"Succès ! Dimensions finales : {df_final.shape}")


# 5. EXPORTATION FINALE

output_file = "../data/processed/dataset_final.csv"
df_final.to_csv(output_file, index=False)
print(f"💾 Dataset sauvegardé : {output_file}")

Suppression de sécurité : 'hrmn' (1440 valeurs uniques)
Suppression de sécurité : 'dep' (107 valeurs uniques)
Suppression de sécurité : 'com' (19211 valeurs uniques)
Suppression de sécurité : 'lat' (157530 valeurs uniques)
Suppression de sécurité : 'long' (158687 valeurs uniques)
Suppression de sécurité : 'voie' (38376 valeurs uniques)
Suppression de sécurité : 'larrout' (150 valeurs uniques)
Succès ! Dimensions finales : (506467, 60)
💾 Dataset sauvegardé : ../data/processed/dataset_final.csv
